<a href="https://colab.research.google.com/github/jeffblackadar/sensation/blob/main/sensory_map_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package installation

In [5]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Install packages
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
!pip install geopandas
!pip install contextily

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgeos-dev is already the newest version (3.6.2-1build2).
libgeos-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
     |████████████████████████████████| 133.1MB 83kB/s 
     |████████████████████████████████| 6.5MB 8.6MB/s 
     |████████████████████████████████| 225kB 38.8MB/s 
  Created wheel for basemap: filename=basemap-1.2.2+dev-cp37-cp37m-linux_x86_64.whl size=121759581 sha256=901ba2bcbf214f789f022a641eb8efc07f375971ec07811fd548fb087be487cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-w8kli9lc/wheels/98/4a/fc/ce719b75d97e646645c225f3332b1b217536100314922e9572
  Created wheel for pyshp: filename=pyshp-2.1.3-cp37-none-any.whl size=37264 sha256=a3aab914905c15270657aa8047b78e2046798247583f782cb6e67e0a85f

     |████████████████████████████████| 1.0MB 9.2MB/s 
     |████████████████████████████████| 15.3MB 376kB/s 
     |████████████████████████████████| 19.1MB 1.2MB/s 


In [17]:
import json
class GenerateLeafletMap:
    def __init__(self,html_file_name):
        self.html_file_name = html_file_name
        self.html_file = open(self.html_file_name, 'w') 
        self.markers = 0
        self.polygons = 0
        self.polylines = 0
        self.layers = 0
        self.type_data = {}
    def close(self):
        self.html_file.close()

    def set_type_data_dict(self,type_data_dict):
        self.type_data = type_data_dict

    def genMapHTMLTop(self):
        self.html_file.write('<!DOCTYPE html>\n')
        self.html_file.write('<html>\n')
        self.html_file.write('<head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n')
        self.html_file.write('<title>map title</title>\n')
        self.html_file.write('<link href="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.5/leaflet.css" rel="stylesheet" />\n')
        self.html_file.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.5/leaflet.js"></script>\n')
        self.html_file.write('<script src="https://code.jquery.com/jquery-2.1.4.min.js"></script>\n')
        self.html_file.write('<style type="text/css">#my-map {\n')
        self.html_file.write('  width:960px;\n')
        self.html_file.write('  height:700px;\n')
        self.html_file.write('}\n')
        self.html_file.write('h1 {\n')
        self.html_file.write('  color: #AA0000;\n')
        self.html_file.write('  font-size: 24px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('h2 {\n')
        self.html_file.write('  color: #000000;\n')
        self.html_file.write('  font-size: 20px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('h3 {\n')
        self.html_file.write('  color: #000000;\n')
        self.html_file.write('  font-size: 16px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')    
        self.html_file.write('p {\n')
        self.html_file.write('  color: #996600;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('li {\n')
        self.html_file.write('  color: #996600;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')  
        self.html_file.write('th {\n')
        self.html_file.write('  text-align: right;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('  color: #663300;\n')
        self.html_file.write('}\n')
        self.html_file.write('td {\n')
        self.html_file.write('  text-align: right;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('  color: #663300;\n')
        self.html_file.write('}\n')
        self.html_file.write('</style>\n')
        self.html_file.write('</head>\n')
        self.html_file.write('<body>\n')

    def genMapHTMLH1(self,map_title):
        self.html_file.write('<h1>'+map_title+'</h1>\n')
        self.html_file.write('<div id="description">description</div>\n')
        self.html_file.write('<div id="latitude">latitude</div>\n')
        self.html_file.write('<div id="longitude">longitude</div>\n')
        self.html_file.write('sight: <div id="sight">sight</div>\n')
        self.html_file.write('hearing: <div id="hearing">hearing</div>\n')
        self.html_file.write('smell: <div id="smell">smell</div>\n')
        self.html_file.write('feeling: <div id="feeling">feeling</div>\n')
                    
    def genMapHTMLScriptTop(self): 
        self.html_file.write('  <div id="my-map"></div>\n')
        self.html_file.write('    <script>\n')
        self.html_file.write('    // This file was generated by this R program:  https://github.com/jeffblackadar/oc-transpo-maps/blob/master/oc_4_gen_html.r\n')
        self.html_file.write('var map;\n')
        self.html_file.write("map = L.map('my-map');\n")
        self.html_file.write('var markers = [];\n')
        self.html_file.write('var polygons = [];\n')
        self.html_file.write('var markers_data = [];\n')
        self.html_file.write('var polygons_data = [];\n')

        self.html_file.write("// Thank you:\n")
        self.html_file.write("// https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula\n")
        self.html_file.write("function distance(lat1, lon1, lat2, lon2) {\n")
        self.html_file.write("  var p = 0.017453292519943295;    // Math.PI / 180\n")  
        self.html_file.write("  var c = Math.cos;\n")
        self.html_file.write("  var a = 0.5 - c((lat2 - lat1) * p)/2 + \n")
        self.html_file.write("          c(lat1 * p) * c(lat2 * p) * \n")
        self.html_file.write("          (1 - c((lon2 - lon1) * p))/2;\n")
        self.html_file.write("  return 12742 * Math.asin(Math.sqrt(a)); // 2 * R; R = 6371 km\n")
        self.html_file.write("}\n")

        self.html_file.write("//https://stackoverflow.com/questions/31790344/determine-if-a-point-reside-inside-a-leaflet-polygon\n")
        self.html_file.write("function isMarkerInsidePolygon(marker, poly) {\n")
        self.html_file.write("    var polyPoints = poly.getLatLngs();       \n")
        self.html_file.write("    var x = marker.getLatLng().lat, y = marker.getLatLng().lng;\n")

        self.html_file.write("    var inside = false;\n")
        self.html_file.write("    for (var i = 0, j = polyPoints.length - 1; i < polyPoints.length; j = i++) {\n")
        self.html_file.write("        var xi = polyPoints[i].lat, yi = polyPoints[i].lng;\n")
        self.html_file.write("        var xj = polyPoints[j].lat, yj = polyPoints[j].lng;\n")

        self.html_file.write("        var intersect = ((yi > y) != (yj > y)) && (x < (xj - xi) * (y - yi) / (yj - yi) + xi);\n")
        self.html_file.write("        if (intersect) inside = !inside;\n")
        self.html_file.write("    }\n")

        self.html_file.write("    return inside;\n")
        self.html_file.write("};\n")

        self.html_file.write("function update_per_proximity(marker) {\n")
        self.html_file.write("    var marker_lat = marker.getLatLng().lat, marker_lon = marker.getLatLng().lng;\n")


        self.html_file.write("// Polygons - check if marker is inside.  If so, light it up.\n")  
        self.html_file.write("    for (cn=0;cn<polygons.length;cn++){\n")
        self.html_file.write("        if(isMarkerInsidePolygon(marker, polygons[cn])==true){\n")
        self.html_file.write("            polygons[cn].setStyle({fillOpacity: 0.7, weight:2,opacity: 0.3});\n")
        self.html_file.write("            document.getElementById('sight').innerHTML = polygons_data[cn].sight;\n")
        self.html_file.write("            document.getElementById('hearing').innerHTML = polygons_data[cn].hearing;\n")
        self.html_file.write("            document.getElementById('feeling').innerHTML = polygons_data[cn].feeling;\n")
        self.html_file.write("            document.getElementById('smell').innerHTML = polygons_data[cn].smell;\n")                                                                  
        self.html_file.write("        }\n")
        self.html_file.write("    }\n")

        self.html_file.write("    for (cn=0;cn<markers.length;cn++){\n")
        self.html_file.write("        dist_to = distance(marker_lat, marker_lon,markers[cn].getLatLng().lat,markers[cn].getLatLng().lng)\n")
        self.html_file.write("        // adjust distance for proximity sensitivity\n")
        self.html_file.write("        if(dist_to < 0.1){\n")
        self.html_file.write("            if(cn<13){\n")
        self.html_file.write("                markers[cn].setIcon(iconRCH);\n")
        self.html_file.write("                markers[cn]._popup.setContent('RCH  <audio controls><source src=\"sounds/rch.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(cn==13){\n")
        self.html_file.write("                markers[cn].setIcon(iconFurnace);\n")
        self.html_file.write("                markers[cn]._popup.setContent('Furnace  <audio controls><source src=\"sounds/furnace.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(cn==14){\n")
        self.html_file.write("                markers[cn].setIcon(iconBusiness);\n")
        self.html_file.write("                markers[cn]._popup.setContent('Business  <audio controls><source src=\"sounds/business.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(cn>14){\n")
        self.html_file.write("                markers[cn].setIcon(iconHouse);\n")
        self.html_file.write("                markers[cn]._popup.setContent('House  <audio controls><source src=\"sounds/house.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")    
        self.html_file.write("        document.getElementById('sight').innerHTML = markers_data[cn].sight;\n")
        self.html_file.write("        document.getElementById('hearing').innerHTML = markers_data[cn].hearing;\n")
        self.html_file.write("        document.getElementById('feeling').innerHTML = markers_data[cn].feeling;\n")
        self.html_file.write("        document.getElementById('smell').innerHTML = markers_data[cn].smell;\n")                                                                  
        self.html_file.write("        } // if dist_to\n")
        self.html_file.write("    } // for\n")

        self.html_file.write("} // function\n")                      
      
        self.html_file.write('var iconHouse = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/house.png",\n')
        self.html_file.write('    //shadowUrl: "leaf-shadow.png",\n')

        self.html_file.write('    iconSize:     [30, 37], // size of the icon\n')
        self.html_file.write('    //shadowSize:   [50, 64], // size of the shadow\n')
        self.html_file.write('    iconAnchor:   [15, 36], // point of the icon which will correspond to markers location\n')
        self.html_file.write('    //shadowAnchor: [4, 62],  // the same for the shadow\n')
        self.html_file.write('    popupAnchor:  [-3, -38] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')

        self.html_file.write('var iconRCH = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/rch.png",\n')
        self.html_file.write('    iconSize:     [35, 33], // size of the icon\n')
        self.html_file.write('    iconAnchor:   [17, 32], // point of the icon which will correspond to \n')
        self.html_file.write('    popupAnchor:  [-3, -34] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')
      
        self.html_file.write('var iconFurnace = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/furnace.png",\n')
        self.html_file.write('    iconSize:     [34, 38], // size of the icon\n')
        self.html_file.write('    iconAnchor:   [17, 37], // point of the icon which will correspond to \n')
        self.html_file.write('    popupAnchor:  [-3, -39] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconBusiness = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/business.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconTravel = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/travel.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconComputer = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/computer.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconQuestion = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/question.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')


                    
    def genMapHTMLMapTop(self):
        self.html_file.write("\n")
                      
        self.html_file.write("      //basemap.fitBounds(geojson.getBounds());\n")
        self.html_file.write("      map.setView([45.468, -75.553], 15);\n")
        self.html_file.write("      basemap.addTo(map);\n")
        self.html_file.write("\n")
        self.html_file.write("      // add legend control layers - global variable with (null, null) allows indiv basemaps and overlays to be added inside functions below\n")
        self.html_file.write("      var controlLayers = L.control.layers(null, null, {\n")
        self.html_file.write('        position: "topleft",\n')
        self.html_file.write("        collapsed: false // false = open by default\n")
        self.html_file.write("      }).addTo(map);\n")
        self.html_file.write("\n")
        #writeLines("      var markers = [];", self$outputFileHtmlCon)
                      

    def genHTMLBaseMap(self):
        self.html_file.write('  window.onload = function() {\n')
        self.html_file.write("    var basemap = L.tileLayer('https://{s}.tile.osm.org/{z}/{x}/{y}.png', {\n")
        self.html_file.write("      attribution: '&copy; "+'<a href="https://osm.org/copyright">OpenStreetMap</a>'+"'\n")
        self.html_file.write("    });\n")

    def genMarker(self,marker_lat, marker_long, marker_type):
        type_data_row = self.type_data[marker_type.lower()]
        type_data_row_json = json.loads(type_data_row)  
        self.html_file.write('  markers.push(L.marker(['+str(marker_lat)+','+str(marker_long)+'], {icon: iconQuestion}).addTo(map));\n')
        self.html_file.write('     markers['+str(self.markers)+"].bindPopup('"+str(marker_type)+"').openPopup();\n")
        self.html_file.write('     markers_data.push('+type_data_row+')\n')
        self.markers = self.markers +1


    def genPolygon(self,polygon_coods_json, polygon_type):
        # Thanks to: https://gis.stackexchange.com/questions/290609/entering-manual-coordinates-to-draw-polygon-in-leaflet
        fill_color="black"
        type_data_row = self.type_data[polygon_type.lower()]
        type_data_row_json = json.loads(type_data_row)

        fill_color = type_data_row_json['color']
        #if(polygon_type.lower()=="hill"):
        #    fill_color="DarkGreen"
        #if(tolower(polygon_type)=="forest"){fill_color="Green"}
        #if(tolower(polygon_type)=="field"){fill_color="Wheat"}
        #if(tolower(polygon_type)=="road"){fill_color="DarkGrey"}
        #if(tolower(polygon_type)=="creek"){fill_color="Blue"}
        
        self.html_file.write('var coords = '+polygon_coods_json+';\n')
                      #writeLines(paste0('var a = JSON.parse(coords);\n')
        self.html_file.write('polygons.push(L.polygon(coords, {\n')
        self.html_file.write('  color: "white",\n')
        self.html_file.write('  fillColor: "'+fill_color+'",\n')
        self.html_file.write('  weight: 0,\n')
        self.html_file.write('  opacity: 0,\n')
        self.html_file.write('  fillOpacity: 0\n')
        self.html_file.write('}).addTo(map));\n')
        self.html_file.write('     polygons['+str(self.polygons)+"].bindPopup('"+polygon_type+"').openPopup();\n")
        self.html_file.write('     polygons_data.push('+type_data_row+')\n')
        self.html_file.write('\n')
        self.polygons = self.polygons +1


    def genPolyline(p1_lat, p1_long, p2_lat, p2_long):
        self.polylines = self.polylines +1
        self.html_file.write('  var pointA = new L.LatLng(',paste0(p1_lat),',',paste0(p1_long),' );\n')
        self.html_file.write('  var pointB = new L.LatLng(',paste0(p2_lat),',',paste0(p2_long),');\n')
        self.html_file.write('  var pointList = [pointA, pointB];\n')
        self.html_file.write('  \n')
        self.html_file.write('  var polyline'+str(self.polylines)+' = new L.Polyline(pointList, {\n')
        self.html_file.write("      color: 'red',\n")
        self.html_file.write('      weight: 3,\n')
        self.html_file.write('      opacity: 0.5,\n')
        self.html_file.write('      smoothFactor: 1\n')
        self.html_file.write('  });\n')
        self.html_file.write('  polyline'+str(self.polylines)+'.addTo(map);\n')

                    

    def genHTMLEndMap(self):
        self.html_file.write("      controlLayers.addOverlay(basemap, 'OpenStreetMap');\n")
        self.html_file.write("      \n")
        self.html_file.write("   // });\n")

        self.html_file.write("// To simulate walking around I made a draggable marker. This will greatly reduce testing time.\n")
        self.html_file.write("// Thanks to\n")
        self.html_file.write("https://stackoverflow.com/questions/57384822/leaflet-detect-when-marker-goes-into-and-out-of-a-circle\n")
        self.html_file.write("var test_marker = new L.marker([45.4672641365588, -75.57309653282166],{\n")
 
        self.html_file.write("    draggable: true,\n")
        self.html_file.write("    autoPan: true\n")
        self.html_file.write("}).addTo(map);\n")
        self.html_file.write("test_marker.setIcon(iconComputer)\n")

        self.html_file.write("test_marker.on('dragend', function (e) {\n")
        self.html_file.write("    document.getElementById('latitude').innerHTML = test_marker.getLatLng().lat;\n")
        self.html_file.write("    document.getElementById('longitude').innerHTML = test_marker.getLatLng().lng;\n")
        self.html_file.write("    document.getElementById('description').innerHTML = distance(test_marker.getLatLng().lat,test_marker.getLatLng().lng,markers[13].getLatLng().lat,markers[13].getLatLng().lng);\n")

        self.html_file.write("    update_per_proximity(test_marker)\n")


        self.html_file.write("});\n")

        self.html_file.write("  };\n")


        self.html_file.write("   // https://gis.stackexchange.com/questions/182068/getting-current-user-location-automatically-every-x-seconds-to-put-on-leaflet\n")
        self.html_file.write("var current_position, current_accuracy;\n")
        self.html_file.write("function onLocationFound(e) {\n")
        self.html_file.write("    // if position defined, then remove the existing position marker and accuracy circle from the map\n")
        self.html_file.write("    if (current_position) {\n")
        self.html_file.write("        map.removeLayer(current_position);\n")
        #writeLines(paste0("       map.removeLayer(current_accuracy);\n")
        self.html_file.write("    }\n")
        self.html_file.write("    var radius = e.accuracy / 2;\n")
        self.html_file.write("    current_position = L.marker(e.latlng).addTo(map)\n")
        self.html_file.write('        .bindPopup("You are within " + radius + " meters from this point");\n')
        self.html_file.write("    current_position.setIcon(iconTravel)\n")
        #writeLines(paste0("current_accuracy = L.circle(e.latlng, radius).addTo(map);\n")
        self.html_file.write("    update_per_proximity(current_position);\n")
        self.html_file.write("}\n")
        self.html_file.write("function onLocationError(e) {\n")
        self.html_file.write("    alert(e.message);\n")
        self.html_file.write("}\n")
        self.html_file.write("map.on('locationfound', onLocationFound);\n")
        self.html_file.write("map.on('locationerror', onLocationError);\n")
        self.html_file.write("// wrap map.locate in a function    \n")
        self.html_file.write("function locate() {\n")
        self.html_file.write("    map.locate({setView: true, maxZoom: 16});\n")
        self.html_file.write("}\n")

        self.html_file.write("// call locate every 3 seconds... forever\n")
        self.html_file.write("setInterval(locate, 3000000);\n")

        self.html_file.write("</script>\n")
                    
    def genHTMLEndBody(self):
        self.html_file.write("<hr>\n")
        #self.html_file.write("<p>Page last modifed: ", format(Sys.Date(),"%d %B, %Y"),"</p>\n")
        self.html_file.write("</body>\n")
        self.html_file.write("</html>\n")

    def genLayer(self,geojsonfile):
        self.layers = self.layers +1
        #print(paste0("To map for ", railYear, ", adding: "))
        #print(output_dbRows_route_style[i_route_style, 1])
        layerName = "layer" + str(self.layers)
                      
        self.html_file.write("\n")
        self.html_file.write("// layer for "+geojsonfile+"\n")
        self.html_file.write("var ",layerName,";\n")
                      
        self.html_file.write("      $.ajax({\n")
        self.html_file.write("        type: 'POST',\n")
        self.html_file.write("        url: '",geojsonfile,"',\n")

        self.html_file.write("        dataType: 'json',\n")
        self.html_file.write("        success: function(response) {\n")
        self.html_file.write("          ",layerName," = L.geoJson(response, {\n")
                      
        self.html_file.write("             onEachFeature: function (feature, layer) {\n")

        self.html_file.write("                 layer.bindPopup('<p>'+feature.properties.RTE_TYPE+'</p><p>'+feature.properties.MODE+'</p>');\n")  
                      
        self.html_file.write("               }\n")
        self.html_file.write("             });\n")
        self.html_file.write("          ",layerName,".setStyle({\n")
        self.html_file.write("            color: 'purple',\n")
        self.html_file.write("            weight: 2,\n")
        self.html_file.write('            opacity: 1\n')
        self.html_file.write("          });\n")
        self.html_file.write("          ",layerName,".addTo(map);\n")
        self.html_file.write("          controlLayers.addOverlay(",layerName,", '<span style=",'"color: purple"',">route</span>');\n")

        self.html_file.write("        }\n")
        self.html_file.write("      });\n")
        self.html_file.write("\n")
                      

gm = GenerateLeafletMap("/content/drive/MyDrive/github/sensation/userloc.html")
gm.genMapHTMLTop()


type_data_dict={
    "hill":'{"color":"brown","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp"}',
    "rch":'{"color":"","sight":"a flat circle","hearing":"wind blowing through trees","smell":"ash","feeling":"cool, damp"}',
    "forest":'{"color":"green","sight":"level terrain, trees","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp"}',
    "field":'{"color":"wheat","sight":"level terrain, grass","hearing":"wind blowing through grass","smell":"grass","feeling":"warm, sun"}',
    "road":'{"color":"gray","sight":"a muddy road","hearing":"squash of footsteps","smell":"mud","feeling":"sun"}',
    "creek":'{"color":"blue","sight":"a running creek","hearing":"water rippling","smell":"fresh water","feeling":"sun"}',
    "business":'{"color":"","sight":"a building","hearing":"horses","smell":"smoke","feeling":"cool, damp"}',
    "furnace":'{"color":"","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp"}',
    "house":'{"color":"","sight":"a house","hearing":"people talking","smell":"wood smoke","feeling":"cool, damp"}'
}
gm.set_type_data_dict(type_data_dict)

gm.genMapHTMLH1("Hopewell, Pennsylvania")
gm.genMapHTMLScriptTop()
gm.genHTMLBaseMap()
gm.genMapHTMLMapTop()

#gm.genMarker(45.3853076228202, -75.7027063624573, '<a href="http://ofnc.ca/programs/fletcher-wildlife-garden">Fletcher Wildlife Garden</a> pollinator garden.')
#gm.genMarker(45.2177207,-76.19668383, '<h1>Location: Naismith Public School Pollinator Garden.<h1><h2>Sponsor: Almonte Horticultural Society</h2>')
#gm.genMarker(45.6769162,-76.7838167, '<h1>Location: Ross Museum in Forester’s Falls</h1><h2>Sponsor: Beachburg Horticultural Society</h2> <a href="https://www.google.com/maps/dir///@45.6769162,-76.7838167,17z">Directions</a>. Beachburg Horticultural Society partnered with the Ross Museum in Forester’s Falls. Plants are in the Heritage Creek-side garden along with several varieties of shrubs and flowers that attract bees and butterflies. Two bee nests have been installed on a nearby building to provide homes for pollen bees. The Allium bulbs of different sizes and colours we planted will attract visitors and pollen bees. This location is often used as a picnic site and ideal for family photos beside a running stream.')
#gm.genMarker(45.1436508,-76.1471922, '<h1>a href="http://www.cpbheritagemuseum.com/">Location: Victoria School Museum gardens</a></h1><h2>Sponsor: Carleton Place Horticultural Society</h2>')
#gm.genMarker(45.3853076228202,-75.7027063624573,'<h3><a href="http://ofnc.ca/programs/fletcher-wildlife-garden">Fletcher Wildlife Garden</a></h3><p></p><a href="https://www.google.com/maps/place/Fletcher+Wildlife+Garden/@45.3870864,-75.7053924,17z/data=!3m1!4b1!4m5!3m4!1s0x4cce05d7e1130193:0x20a2dc5c1a64075f!8m2!3d45.3870864!4d-75.7032037">Directions from Google Maps</a>')

#gm.genMarker(45.2177207,-76.19668383,'<h3>Naismith Public School Pollinator Garden</h3><p>Sponsor: Almonte Horticultural Society</p><p>260 King St, Almonte, ON K0A 1A0<a href="https://www.google.com/maps/place/Naismith+Memorial+Public+School/@45.2177207,-76.1966838,17z/data=!3m1!4b1!4m5!3m4!1s0x4cd21a82b9e49ee7:0x63ecf7f1f28b0063!8m2!3d45.2177207!4d-76.1944951"><br>Directions from Google Maps</a><p>')
#gm.genMarker(45.6769162,-76.7838167,'<h3>Ross Museum</h3><p>Beachburg Horticultural Society partnered with the Ross Museum in Forester’s Falls. Plants are in the Heritage Creek-side garden along with several varieties of shrubs and flowers that attract bees and butterflies. Two bee nests have been installed on a nearby building to provide homes for pollen bees. The Allium bulbs of different sizes and colours we planted will attract visitors and pollen bees. This location is often used as a picnic site and ideal for family photos beside a running stream.</p><p>2022 Foresters Falls Rd, Foresters Falls, ON K0J 1V0<a href="https://www.google.com/maps/dir///@45.6769162,-76.7838167,17z"><br>Directions from Google Maps</a><p>')
#gm.genMarker(45.1436508,-76.1471922,'<h3><a href="http://www.cpbheritagemuseum.com/">Victoria School Museum</a></h3><p></p><p>267 Edmund St, Carleton Place, ON K7C 3E8<a href=""><br>Directions from Google Maps</a><p>')
#gm.genMarker(45.3853076228202,-75.7027063624573,'<h3><a href="http://ofnc.ca/programs/fletcher-wildlife-garden">Fletcher Wildlife Garden</a></h3><p></p><p>near 865, Prince of Wales Dr, Ottawa, ON K2C 3K1<a href="https://www.google.com/maps/place/Fletcher+Wildlife+Garden/@45.3870864,-75.7053924,17z/data=!3m1!4b1!4m5!3m4!1s0x4cce05d7e1130193:0x20a2dc5c1a64075f!8m2!3d45.3870864!4d-75.7032037"><br>Directions from Google Maps</a><p>')
#gm.genMarker(45.5364896,-77.0975428,'<h3>The Eganville and Area Horticultural Society&apos;s Pollinator Garden</h3><p>The Eganville and Area Horticultural Society&apos;s Pollinator Garden is located on a plot of land generously donated by the township, located adjacent to the Municipal Water Treatment Center at 401 John Street, Eganville. It is clearly marked by a tall Garden Ontario OHA Pollinator Haven sign. Stepping stones inside the garden will help you go through and enjoy the many native species including, chokecherry, bark shrubs, serviceberry trees, cosmos, coneflowers, beebalms, sedums, thyme and asters. The thriving plant life is anchored by 3 pear trees. Pollinators also benefit from aged logs with drilled holes that provide natural bee habitats. Planting began in 2015, and the maturing garden is now abundant, and continues to buzz with activity throughout growing seasons.</p><p>85 John St, Eganville, ON K0J 1T0<a href="https://www.google.com/maps/place/John+Egan+Geo-heritage+Walking+Trail/@45.5364896,-77.0975428,17z/data=!4m12!1m6!3m5!1s0x4cd15eb8d8166041:0x73d40ecc420818ae!2sJohn+Egan+Geo-heritage+Walking+Trail!8m2!3d45.5364671!4d-77.0954829!3m4!1s0x4cd15eb8d8166041:0x73d40ecc420818ae!8m2!3d45.5364671!4d-77.0954829"><br>Directions from Google Maps</a><p>')
#gm.genPolyline(45.4376105533903, -75.6575202941895,45.8376105533903, -75.8575202941895)

gm.genMarker(45.48069524143359, -75.55294256515234,'RCH')
gm.genMarker(45.479835774616625, -75.54924580025349,'RCH')
gm.genMarker(45.48087461798589, -75.54745352258493,'RCH')
gm.genMarker(45.48159160526861, -75.54374033081376,'RCH')
gm.genMarker(45.48074292337904, -75.54225039323578,'RCH')
gm.genMarker(45.481173812195195, -75.54037418908419,'RCH')
gm.genMarker(45.481570547824525, -75.53939571220832,'RCH')
gm.genMarker(45.47208041856893, -75.53911713843256,'RCH')
gm.genMarker(45.47212550944454, -75.53739164423271,'RCH')
gm.genMarker(45.47153426016818, -75.53834179690928,'RCH')
gm.genMarker(45.47052250587142, -75.53878836432347,'RCH')
gm.genMarker(45.47107363424301, -75.53987385471375,'RCH')
gm.genMarker(45.46921990642112, -75.54531261117356,'RCH')
gm.genMarker(45.47816607361546, -75.54191997834484,'Furnace')
gm.genMarker(45.477661000000005, -75.536143,'Business')
gm.genMarker(45.47630702214738, -75.53642835143401,'House')
gm.genMarker(45.475436884379604, -75.53662901664491,'House')
gm.genMarker(45.47467801918609, -75.53620538296144,'House')
gm.genMarker(45.474633279317196, -75.53493099286987,'House')
gm.genMarker(45.473971523461664, -75.53602143272323,'House')
gm.genMarker(45.47307535351896, -75.53460232148552,'House')
gm.genMarker(45.47537087556559, -75.53856218959736,'House')
gm.genMarker(45.47484413276825, -75.54051028187385,'House')
gm.genMarker(45.4742765520621, -75.54142519878856,'House')
gm.genPolygon('[[45.47241415488257,-75.53572719742755],[45.47238935371979,-75.53569352210094],[45.47238935371979,-75.53569352210094],[45.47238935371979,-75.53569352210094],[45.47234085901102,-75.53569509718041],[45.47234085901102,-75.53569509718041],[45.47197604106905,-75.53563798499944],[45.46805269997154,-75.5421093780862],[45.4682206938845,-75.54958569802682],[45.46955257782138,-75.55095641199402],[45.46987837516826,-75.54856686042025],[45.47318486702365,-75.54142606215255],[45.47380079101707,-75.537475208911],[45.47241415488257,-75.53572719742755]]','hill')
gm.genPolygon('[[45.47214078761794,-75.53532230780189],[45.47221352967083,-75.53531994469992],[45.47218928231977,-75.53532073240127],[45.47218928231977,-75.53532073240127],[45.47218928231977,-75.53532073240127],[45.47729895024767,-75.53625820894537],[45.47734412298491,-75.53604983855001],[45.47218706652228,-75.53518288151251],[45.47214078761794,-75.53532230780189]]','road')
gm.genPolygon('[[45.47580823176229,-75.53558247203736],[45.47704970354474,-75.53433521254571],[45.47693552785559,-75.53326992955068],[45.47687354597413,-75.53092705595019],[45.47634435501526,-75.52970288970072],[45.47572205340078,-75.52872316386939],[45.47233144265936,-75.53510922931032],[45.47580823176229,-75.53558247203736]]','field')
gm.genPolygon('[[45.47650015207343,-75.55611204320581],[45.4759563020478,-75.5554743330175],[45.47578835159154,-75.55406593468511],[45.47513817652603,-75.55284547143914],[45.47412043355972,-75.55139549698119],[45.4742395551631,-75.54973653313603],[45.47372876855899,-75.54813236136216],[45.47402527971506,-75.54695041807496],[45.47385556963202,-75.54543870321007],[45.47430194596157,-75.54452775292533],[45.474693202849366,-75.54320478374017],[45.47563385469154,-75.54134676971047],[45.475564925972925,-75.54007313910543],[45.47581064597427,-75.53875482570241],[45.47610429194899,-75.53740046174431],[45.47585628474507,-75.53706367986828],[45.47563538502541,-75.53841568036472],[45.47543208491983,-75.5393533068027],[45.475231538822236,-75.54046324969228],[45.474513542252495,-75.54258991717792],[45.474192829687574,-75.54377267829486],[45.47370788174085,-75.54530554785137],[45.47365276130221,-75.54641073741054],[45.47348959380013,-75.54834696436754],[45.473952431806076,-75.54998715778059],[45.47392864962624,-75.5515395982866],[45.474872002733555,-75.55288851663674],[45.47552765818474,-75.55445362442379],[45.47574026529492,-75.55561920381696],[45.47623835671419,-75.55643079737537],[45.47650015207343,-75.55611204320581]]','creek')
gm.genPolygon('[[45.480328664562556,-75.55429929748178],[45.48416668061128,-75.53941501180533],[45.48214861697988,-75.53913556406698],[45.478760533807375,-75.5381418956489],[45.47846699251775,-75.54404827994126],[45.479112290192724,-75.54647584644002],[45.47825694032927,-75.54912427642176],[45.47884814306308,-75.55120878393257],[45.479822331144774,-75.55297064818701],[45.480328664562556,-75.55429929748178]]','forest')


gm.genHTMLEndMap()
gm.genHTMLEndBody()

gm.close()

# Test generated map using http server on Google Colab

In [18]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)")+"drive/MyDrive/github/sensation/userloc.html")

https://e5nji3clea-496ff2e9c6d22116-8000-colab.googleusercontent.com/drive/MyDrive/github/sensation/userloc.html


In [19]:
!python -m http.server 8000

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [23/Apr/2021 02:01:24] "GET /drive/MyDrive/github/sensation/userloc.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 02:01:25] "GET /drive/MyDrive/github/sensation/icons/question.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 02:01:25] "GET /drive/MyDrive/github/sensation/icons/computer.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 02:01:25] code 404, message File not found
127.0.0.1 - - [23/Apr/2021 02:01:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Apr/2021 02:01:39] "GET /drive/MyDrive/github/sensation/icons/house.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 02:01:40] "GET /drive/MyDrive/github/sensation/icons/house.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 02:01:44] "GET /drive/MyDrive/github/sensation/icons/house.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 02:01:45] "GET /drive/MyDrive/github/sensation/icons/rch.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 02:01:47] "GET /drive/MyDriv

# Relocate map coordinates to get ready to unroll map at your location's latitude longitude

In [11]:
# Input the location to unroll the map at.
local_lat = 45.475871
local_lon = -75.545315

local_lat = 45.477661
local_lon = -75.536143

In [13]:
shapefiles_path = "/content/drive/MyDrive/github/sensation/shapefiles/"
from geographiclib.geodesic import Geodesic
import geopy
from geopy.distance import VincentyDistance
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os

def transition_point(origin_lat, origin_lon, dest_lat, dest_lon, new_map_origin_lat, new_map_origin_lon):
    #print(origin_lat, origin_lon, dest_lat, dest_lon, new_map_origin_lat, new_map_origin_lon)  
    geod = Geodesic.WGS84.Inverse(origin_lat, origin_lon, dest_lat, dest_lon)
    distance = geod['s12']
    bearing = geod['azi1']
    origin = geopy.Point(new_map_origin_lat, new_map_origin_lon)
    destination = VincentyDistance(meters=distance).destination(origin, bearing)
    lat2, lon2 = destination.latitude, destination.longitude
    return(lat2, lon2)


geotif_crs = 4326
new_map_shp = gpd.GeoDataFrame()
new_map_shp['geometry'] = None
new_map_shp.crs = ("EPSG:" + str(geotif_crs))
new_map_shp.geometry = new_map_shp.geometry.to_crs(crs=geotif_crs)
new_map_shp.to_crs(crs=geotif_crs)
new_map_shp = new_map_shp.to_crs(epsg=geotif_crs)


h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements.shp"))
for index, row in h_df.iterrows():
    #print(row['type'], row['name'], row['geometry'])
    p = row['geometry']
    lat2, lon2 = transition_point(40.20595294149071, -75.77220799054444, p.y, p.x, local_lat, local_lon)

    point = Point(lon2, lat2)
    new_p_row = {'id':row['id'], 'geometry':point, 'type':row['type'], 'name':row['name']}
    print("gm.genMarker("+str(lat2)+", "+str(lon2)+",'"+str(row['type'])+"')")
    new_map_shp = new_map_shp.append(new_p_row, ignore_index=True)

new_map_shp.to_file(os.path.join(shapefiles_path,"new_map.shp"))


new_map_shp_poly = gpd.GeoDataFrame()
new_map_shp_poly['geometry'] = None
new_map_shp_poly.crs = ("EPSG:" + str(geotif_crs))
new_map_shp_poly.geometry = new_map_shp_poly.geometry.to_crs(crs=geotif_crs)
new_map_shp_poly.to_crs(crs=geotif_crs)
new_map_shp_poly = new_map_shp_poly.to_crs(epsg=geotif_crs)
polygon_points_array = []

h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements_polys.shp"))
for index, row in h_df.iterrows():
    poly = row['geometry']
    polygon_points = ""
    comma = ""
    for p in poly.exterior.coords:
        lat2, lon2 = transition_point(40.20595294149071, -75.77220799054444, p[1], p[0], local_lat, local_lon)
        polygon_points = polygon_points + comma + "["+str(lat2)+","+str(lon2)+"]"
        comma = ","
        #print(lat2, lon2)
    #/content/drive/MyDrive/crane_obs/crane_hopewell/hopewell_selected_elements.shp
        point = Point(lon2, lat2)
        polygon_points_array.append(point)
    polygon = Polygon(polygon_points_array)
    new_p_row = {'id':row['id'], 'geometry':polygon, 'type':row['type'], 'name':row['name']}
    print("gm.genPolygon('["+polygon_points+"]','"+str(row['type'])+"')")
    new_map_shp_poly = new_map_shp_poly.append(new_p_row, ignore_index=True)
print("")
new_map_shp_poly.to_file(os.path.join(shapefiles_path,"new_map_polys.shp"))



gm.genMarker(45.48069524143359, -75.55294256515234,'RCH')
gm.genMarker(45.479835774616625, -75.54924580025349,'RCH')
gm.genMarker(45.48087461798589, -75.54745352258493,'RCH')
gm.genMarker(45.48159160526861, -75.54374033081376,'RCH')
gm.genMarker(45.48074292337904, -75.54225039323578,'RCH')
gm.genMarker(45.481173812195195, -75.54037418908419,'RCH')
gm.genMarker(45.481570547824525, -75.53939571220832,'RCH')
gm.genMarker(45.47208041856893, -75.53911713843256,'RCH')
gm.genMarker(45.47212550944454, -75.53739164423271,'RCH')
gm.genMarker(45.47153426016818, -75.53834179690928,'RCH')
gm.genMarker(45.47052250587142, -75.53878836432347,'RCH')
gm.genMarker(45.47107363424301, -75.53987385471375,'RCH')
gm.genMarker(45.46921990642112, -75.54531261117356,'RCH')
gm.genMarker(45.47816607361546, -75.54191997834484,'Furnace')
gm.genMarker(45.477661000000005, -75.536143,'Business')
gm.genMarker(45.47630702214738, -75.53642835143401,'House')
gm.genMarker(45.475436884379604, -75.53662901664491,'House')
gm.g

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import geopandas as gpd
import pandas as pd
import contextily as ctx

print("Original Map")
h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements.shp"))

#print(h_df.crs)
# Change crs to one compatible with basemap
h_df = h_df.to_crs(epsg=3857) #3857

ax = h_df.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, zoom=15)

h_df['coords'] = h_df['geometry'].apply(lambda x: x.representative_point().coords[:])
h_df['coords'] = [coords[0] for coords in h_df['coords']]
for idx, row in h_df.iterrows():
    plt.annotate(s=row['type'], xy=row['coords'], horizontalalignment='center')

print("Unrolled map at your location")
new_map_df = gpd.read_file(os.path.join(shapefiles_path,"new_map.shp"))

print(new_map_df.crs)
# Change crs to one compatible with basemap
new_map_df = new_map_df.to_crs(epsg=3857) #3857
print(new_map_df.crs)

ax = new_map_df.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, zoom=13)

new_map_df['coords'] = new_map_df['geometry'].apply(lambda x: x.representative_point().coords[:])
new_map_df['coords'] = [coords[0] for coords in new_map_df['coords']]
for idx, row in new_map_df.iterrows():
    plt.annotate(s=row['type'], xy=row['coords'], horizontalalignment='center')

new_map_df = gpd.read_file(os.path.join(shapefiles_path,"new_map_polys.shp"))
#new_map_df.boundary.plot(ax=ax,color="red",alpha=0.3)